# Import Libraries

In [17]:
# Pandas & Numpy 
import pandas as pd
import numpy as np

# Tensorflow
import tensorflow as tf
import tensorflow_recommenders as tfrs

tf.compat.v1.enable_eager_execution()

# Types
from typing import Dict, Text

# System
import os 
import pprint
import tempfile

# Loading & Reading Data

In [33]:
# Read original Data to collect all columns
original_df  = pd.read_csv('../../00_Data/Rec_Sys_precleaned.csv', index_col=0)

In [34]:
# Check Datatypes to identify column types that need to be changed
original_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245898 entries, 0 to 272403
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   InvoiceNo         245898 non-null  int64  
 1   StockCode         245898 non-null  object 
 2   Quantity          245898 non-null  int64  
 3   InvoiceDate       245898 non-null  object 
 4   DeliveryDate      245898 non-null  object 
 5   Discount%         245898 non-null  float64
 6   ShipMode          245898 non-null  object 
 7   ShippingCost      245898 non-null  float64
 8   CustomerID        245898 non-null  int64  
 9   Gender            245898 non-null  object 
 10  Age               245898 non-null  int64  
 11  Income            245898 non-null  object 
 12  Zipcode           245898 non-null  int64  
 13  Customer Segment  245898 non-null  object 
 14  Product Name      245898 non-null  object 
 15  Description       245898 non-null  object 
 16  Category          24

In [20]:
# CustomerID

In [36]:
# Convert CustomerID to String
original_df['CustomerID']  = original_df['CustomerID'].astype(str)
#original_df['Age'] = original_df['Age'].astype(str)


# Selection of Features

In [37]:
# First get the list of features
original_df.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'DeliveryDate',
       'Discount%', 'ShipMode', 'ShippingCost', 'CustomerID', 'Gender', 'Age',
       'Income', 'Zipcode', 'Customer Segment', 'Product Name', 'Description',
       'Category', 'Brand', 'Unit Price', 'Num_word_text', 'Num_word_cat'],
      dtype='object')

In [38]:
# Select important Features
interactions_dict = original_df.groupby(['CustomerID','StockCode', 'Age', 'Gender', 'Product Name', 'Description']).agg({'CustomerID': 'count'})
interactions_dict.columns = ['purchased']
interactions_dict.reset_index(inplace=True)
# Set all purchases to 1 
interactions_dict['purchased'] = 1 

# Transform Table into a Dictionary, which will be fed into tensor slices
interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()} 
interactions = tf.data.Dataset.from_tensor_slices(interactions_dict)

## map the features in interactions
interactions = interactions.map(lambda x: {
    'user_id' : x['CustomerID'], 
    'product_id' : x['StockCode'], 
    'purchased' : float(x['purchased']),
    'age' : x['Age'],
    'gender' : x['Gender'],
    'product_name' : x['Product Name'],
    'description' : x['Description']
})

In [64]:
# We do a similar step for items, where this is the reference table for items to be recommended 
items_dict = original_df[['StockCode']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(items_dict)
items = items.map(lambda x: x['StockCode'])

In [51]:
# Prepare feature Vocabularies
unique_item_titles = np.unique(np.concatenate(list(interactions.batch(1000).map(lambda x: x['product_id']))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000).map(lambda x: x["user_id"]))))

2023-07-06 16:16:07.616399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [172701]
	 [[{{node Placeholder/_2}}]]
2023-07-06 16:16:10.643575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_6}}]]


# Feature Preparation

## Categorical Features

### Product ID 

In [26]:
# Define Vocabulary for Product ID 
product_id_lookup = tf.keras.layers.StringLookup()

# Fill the Vocabulary with Data
product_id_lookup.adapt(interactions.map(lambda x: x['product_id']))

2023-07-06 15:48:12.882902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]


In [27]:
# Define Embeddings for Product ID
product_id_embedding = tf.keras.layers.Embedding(
    input_dim=product_id_lookup.vocab_size(),
    output_dim=32
)

In [28]:
# Put embedding & lookup into a single layer
product_id_model = tf.keras.Sequential([product_id_lookup, product_id_embedding])

In [30]:
# Print a product ID for test
original_df['StockCode'][1]

'71053'

In [32]:
# Get the embedding of a Product 
product_id_model(['71053'])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.011115  , -0.02418478, -0.0098845 , -0.01271111,  0.02693986,
         0.01822654,  0.04377073, -0.02430462, -0.01901107, -0.02767054,
         0.03339065, -0.02163121,  0.0336875 , -0.03392986, -0.04298409,
         0.02738979, -0.04508037, -0.02172426,  0.04127396, -0.0091097 ,
        -0.03371358, -0.0465629 , -0.00711776,  0.03578179,  0.01007149,
         0.04761975, -0.02998685,  0.03404905,  0.03231731,  0.04661078,
        -0.01793783, -0.04587294]], dtype=float32)>

### User ID

In [39]:
# Define Vocabulary for User ID 
user_id_lookup = tf.keras.layers.StringLookup()

# Fill the Vocabulary with Data
user_id_lookup.adapt(interactions.map(lambda x: x['user_id']))

2023-07-06 15:55:01.589461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]


In [40]:
# Define Embeddings for User ID
user_id_embedding = tf.keras.layers.Embedding(
    input_dim=user_id_lookup.vocab_size(),
    output_dim=32
)

In [41]:
# Put embedding & lookup into a single layer
user_id_model = tf.keras.Sequential([user_id_lookup, user_id_embedding])

In [42]:
# Test Embedding
user_id_model(['1'])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.02445397, -0.04554048,  0.00478823, -0.01400031, -0.03067191,
         0.02913086, -0.02198067,  0.03065082, -0.00668854, -0.02594599,
         0.00353013,  0.0288906 ,  0.02416774,  0.03861767, -0.02395483,
        -0.0424164 ,  0.01457774,  0.01994163,  0.01415397, -0.04684038,
        -0.0397626 ,  0.00553857,  0.0146699 ,  0.04277406,  0.00418593,
        -0.03344502, -0.03156396,  0.00209828,  0.03903531, -0.02915115,
        -0.03050636,  0.00424571]], dtype=float32)>

### Gender

In [43]:
# Define Vocabulary
gender_lookup = tf.keras.layers.StringLookup()

# Fill Vocabulary with Data
gender_lookup.adapt(interactions.map(lambda x: x['gender']))

# Create Embeddings for Gender
gender_embedding = tf.keras.layers.Embedding(gender_lookup.vocab_size(), 32)

# Create model
gender_model = tf.keras.Sequential([gender_lookup, gender_embedding])

2023-07-06 15:59:05.325098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [172701]
	 [[{{node Placeholder/_2}}]]


In [44]:
# Print a product ID for test
original_df['Gender'][1]

'female'

In [45]:
gender_model(['female'])

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.01782995,  0.03942413,  0.02142593, -0.04416292, -0.00846276,
         0.01270528,  0.0266862 , -0.02819345, -0.04914814, -0.04015294,
        -0.03586718, -0.04147746,  0.02575041, -0.01331297,  0.01451247,
         0.04831688, -0.017432  , -0.02092351, -0.03603212,  0.04513839,
         0.01373551,  0.01329667,  0.04933478,  0.02328858, -0.03699573,
        -0.04156264,  0.0205009 , -0.02418802, -0.04335693,  0.03123492,
         0.02103129, -0.03494483]], dtype=float32)>

## Numeric Features

### Age

In [46]:
# Check Data Type of Age as Sanity Check
original_df.Age.info()

<class 'pandas.core.series.Series'>
Int64Index: 245898 entries, 0 to 272403
Series name: Age
Non-Null Count   Dtype
--------------   -----
245898 non-null  int64
dtypes: int64(1)
memory usage: 11.8 MB


In [48]:
# Standardization of Age

# Set up Object
age_normalization = tf.keras.layers.Normalization(
    axis=None
)

# Fill with Data
age_normalization.adapt(interactions.map(lambda x: x['age']).batch(1024))


2023-07-06 16:06:06.844786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_0}}]]


In [83]:
# Discretization of Age
max_age = interactions.map(lambda x: x['age']).reduce(
    tf.cast(0, tf.int64), tf.maximum).numpy().max()
min_age = interactions.map(lambda x: x['age']).reduce(
    np.int64(1e9), tf.minimum).numpy().min()

age_buckets = np.linspace(
    min_age, max_age, num=1000
) 

2023-07-06 17:14:41.396452: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]
2023-07-06 17:14:47.238989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [172701]
	 [[{{node Placeholder/_0}}]]


## Text Features

### Product Name

In [49]:
# Vectorization for Product Name
product_text = tf.keras.layers.TextVectorization()
product_text.adapt(interactions.map(lambda x: x['product_name']))

2023-07-06 16:07:08.314583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]


### Product Description

In [50]:
# Vectorization for Product Description
product_description = tf.keras.layers.TextVectorization()
product_description.adapt(interactions.map(lambda x: x['description']))

2023-07-06 16:08:35.605004: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [172701]
	 [[{{node Placeholder/_1}}]]


# Model Definition

## Query Model

In [132]:
class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_id_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids)+1,32)
        ])
        
        self.gender_embedding = tf.keras.Sequential([
            gender_lookup,
            tf.keras.layers.Embedding(gender_lookup.vocab_size(),32)
        ])
        
        self.age_embeding = tf.keras.Sequential([
            tf.keras.layers.Discretization(age_buckets.tolist()),
            tf.keras.layers.Embedding(len(age_buckets) +1, 32)
        ])
        
    def call(self, inputs):
        return tf.concat([
            self.user_id_embedding(inputs['user_id']),
            self.gender_embedding(inputs['gender']),
            self.age_embeding(inputs['age']),
        ], axis=1)

## Candidate Model

In [133]:
class ItemModel(tf.keras.Model): 
    def __init__(self):
        super().__init__()
        
        max_tokens = 10_000
        # Product ID  
        self.product_id_embedding = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary=unique_item_titles,mask_token=None),
            tf.keras.layers.Embedding(len(unique_item_titles) + 1, 32)
        ])
        
        # Product Name 
        self.product_name_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens
        )
        self.product_name_embedding = tf.keras.Sequential([
            self.product_name_vectorizer,
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            tf.keras.layers.GlobalAveragePooling1D()
        ]) 
        self.product_name_vectorizer.adapt(items)
        
        # Product Description
        self.product_desc_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens
        ) 
        self.product_desc_embedding = tf.keras.Sequential([
            self.product_desc_vectorizer,
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            tf.keras.layers.GlobalAveragePooling1D()
        ])
        self.product_desc_vectorizer.adapt(items)
        
    def call(self, titels):
        return tf.concat([
            self.product_id_embedding(titels),
            self.product_name_embedding(titels),
            self.product_desc_embedding(titels)
        ], axis=1) 

## Combined Model

In [134]:
class RetailModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.query_model = tf.keras.Sequential([
            UserModel(),
            tf.keras.layers.Dense(32)
        ])
        self.candidate_model = tf.keras.Sequential([
            ItemModel(),
            tf.keras.layers.Dense(32)
        ]) 
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=items.batch(128).map(self.candidate_model),
            ),
        )
        
    def compute_loss(self, features, training=False):
        query_embeddings = self.query_model({
            "user_id" : features['user_id'],
            "gender" : features['gender'],
            "age" : features['age'],
        })
        item_embeddings = self.candidate_model({
            "product_id" : features['product_id'],
            "product_name" : features['product_name'],
            "product_description" : features['description'],
        })
        
        return self.task(query_embeddings, item_embeddings)

# Train Test Split

In [135]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = interactions.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(60_000)
test = shuffled.skip(60_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

# Building the Model

In [136]:
model = RetailModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

Epoch 1/3


2023-07-06 17:53:03.488266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [172701]
	 [[{{node Placeholder/_5}}]]
2023-07-06 17:53:03.488702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [172701]
	 [[{{node Placeholder/_2}}]]


2023-07-06 17:53:07.097162: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Placeholder/_7}}]]


 1/30 [>.............................] - ETA: 1:53 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0264 - factorized_top_k/top_100_categorical_accuracy: 0.0552 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-06 17:53:07.453617: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]


...

2023-07-06 17:53:37.151438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype resource
	 [[{{node Placeholder/_22}}]]


28/30 [===========================>..] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-06 17:53:37.429347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype resource
	 [[{{node Placeholder/_22}}]]


30/30 [==============================] - 11s 284ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan


2023-07-06 17:53:37.780379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]


# Evaluate Model

In [137]:
train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

2023-07-06 17:53:40.948285: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]


 2/30 [=>............................] - ETA: 6s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan  

2023-07-06 17:53:41.187688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64
	 [[{{node Placeholder/_10}}]]


 3/30 [==>...........................] - ETA: 5s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-06 17:53:41.403829: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64
	 [[{{node Placeholder/_12}}]]
2023-07-06 17:53:41.617447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]


...

3/5 [=================>............] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-06 17:53:51.297583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype resource
	 [[{{node Placeholder/_13}}]]


4/5 [=======================>......] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan

2023-07-06 17:53:51.775771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64
	 [[{{node Placeholder/_12}}]]


5/5 [==============================] - 5s 417ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: nan - regularization_loss: 0.0000e+00 - total_loss: nan
Top-100 accuracy (train): 0.00.
Top-100 accuracy (test): 0.00.


# Predictions

In [138]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model, k=10)

index.index_from_dataset(items.batch(100).map(lambda title: (title, model.candidate_model(title))))

2023-07-06 17:53:52.158526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64
	 [[{{node Placeholder/_12}}]]


# Create Top K List per User

In [143]:
# First we select all users in a list 
all_users = original_df['CustomerID'].unique().tolist()

In [ ]:
# Create Data Frame 
# First Set up empty lists 
c_name = []
i_name = []

# Iterate through all users 
for u in all_users:
    for i in range(10):
        ranked_item = index(np.array([u]))[1][0][i].numpy()
        c_name.append(u)
        i_name.append(ranked_item)


# Create Dataframe         
df_predictions = pd.DataFrame({
    "CustomerID": c_name,
    "StockCode" : i_name
})

# Add Predictions Column
df_predictions['predictions'] = 1.0

In [ ]:
# Create Test Dataset
test_list = list(test.as_numpy_iterator())
df_test = pd.DataFrame(test_list, columns=['user_id', 'product_id', 'purchased'])
df_test['user_id'] = df_test['user_id'].str.decode('utf-8')
df_test['product_id'] = df_test['product_id'].str.decode('utf-8')
df_test = df_test.rename(columns={'user_id':'CustomerID', 'product_id':'StockCode'})


In [ ]:
# Ensure that the same items and users are in both sets 
# Find the set of unique items in both the train and test sets
prediction_items = set(df_predictions['CustomerID'].unique())
test_items = set(df_test['CustomerID'].unique())

# Find the intersection of the sets from step 1
common_items = prediction_items.intersection(test_items)

# Filter the train and test sets to include only the rows with item IDs that are in the intersection set
df_predictions = df_predictions[df_predictions['CustomerID'].isin(common_items)]
df_test = df_test[df_test['CustomerID'].isin(common_items)]

In [ ]:
# Send to csv for further assessment
df_predictions.to_csv('pred_hybrid.csv',index=False)
df_test.to_csv('test_hybrid.csv', index=False)